<a href="https://colab.research.google.com/github/sviatmykytyn/ab-test-portfolio-project/blob/main/ab_test_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-bigquery > install_log.txt

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from google.colab import drive

# Connecting and SQL Query

In [ ]:
# Connecting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Changing work folder
%cd /content/drive/MyDrive/PortfolioTask

/content/drive/MyDrive/PortfolioTask


In [ ]:
# Authenticate user to access Google Cloud
auth.authenticate_user()

# Create a client for accessing BigQuery
client = bigquery.Client(project="data-analytics-mate")

# SQL Query
query = """
WITH session_info AS (
SELECT s.date,
       s.ga_session_id,
       sp.country,
       sp.device,
       sp.continent,
       sp.channel,
       ab.test,
       ab.test_group
FROM `DA.ab_test` ab
JOIN `DA.session` s
ON ab.ga_session_id = s.ga_session_id
JOIN `DA.session_params` sp
ON sp.ga_session_id = ab.ga_session_id
),
session_with_orders AS (
SELECT session_info.date,
       session_info.country,
       session_info.device,
       session_info.continent,
       session_info.channel,
       session_info.test,
       session_info.test_group,
       COUNT(DISTINCT o.ga_session_id) AS session_with_orders
FROM `DA.order` o
JOIN session_info
ON o.ga_session_id = session_info.ga_session_id
GROUP BY 1, 2, 3, 4, 5, 6, 7
),
events AS (
SELECT session_info.date,
       session_info.country,
       session_info.device,
       session_info.continent,
       session_info.channel,
       session_info.test,
       session_info.test_group,
       ep.event_name,
       COUNT(ep.ga_session_id) AS event_cnt
FROM `DA.event_params` ep
JOIN session_info
ON ep.ga_session_id = session_info.ga_session_id
GROUP BY 1, 2, 3, 4, 5, 6, 7, 8
),
session AS (
SELECT session_info.date,
       session_info.country,
       session_info.device,
       session_info.continent,
       session_info.channel,
       session_info.test,
       session_info.test_group,
       COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
FROM session_info
GROUP BY 1, 2, 3, 4, 5, 6, 7
),
account AS (
SELECT session_info.date,
       session_info.country,
       session_info.device,
       session_info.continent,
       session_info.channel,
       session_info.test,
       session_info.test_group,
       COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
FROM `DA.account_session` acs
JOIN session_info
ON acs.ga_session_id = session_info.ga_session_id
GROUP BY 1, 2, 3, 4, 5, 6, 7
)

SELECT session_with_orders.date,
       session_with_orders.country,
       session_with_orders.device,
       session_with_orders.continent,
       session_with_orders.channel,
       session_with_orders.test,
       session_with_orders.test_group,
       "session with orders" AS event_name,
       session_with_orders.session_with_orders AS value
FROM session_with_orders
UNION ALL
SELECT events.date,
       events.country,
       events.device,
       events.continent,
       events.channel,
       events.test,
       events.test_group,
       event_name,
       events.event_cnt AS value
FROM events
UNION ALL
SELECT session.date,
       session.country,
       session.device,
       session.continent,
       session.channel,
       session.test,
       session.test_group,
       "session" AS event_name,
       session.session_cnt AS value
FROM session
UNION ALL
SELECT account.date,
       account.country,
       account.device,
       account.continent,
       account.channel,
       account.test,
       account.test_group,
       "new account" AS event_name,
       account.new_account_cnt AS value
FROM account
"""

query_job = client.query(query) # Execute SQL query
results = query_job.result() # Waiting for the query to complete

# Convert the query results to a DataFrame
df = results.to_dataframe()

# Show the result
df

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Kenya,desktop,Africa,Organic Search,2,1,session with orders,1
1,2020-11-01,Puerto Rico,desktop,Americas,Organic Search,2,2,session with orders,1
2,2020-11-02,Bahrain,desktop,Asia,Undefined,2,2,session with orders,1
3,2020-11-02,New Zealand,desktop,Oceania,Organic Search,2,1,session with orders,1
4,2020-11-03,Croatia,desktop,Europe,Paid Search,2,1,session with orders,1
...,...,...,...,...,...,...,...,...,...
800991,2020-11-28,Vietnam,desktop,Asia,Paid Search,2,1,view_promotion,1
800992,2020-12-16,Vietnam,mobile,Asia,Paid Search,3,2,first_visit,1
800993,2020-12-31,Vietnam,desktop,Asia,Direct,4,2,user_engagement,1
800994,2020-11-04,Vietnam,desktop,Asia,Direct,1,1,begin_checkout,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  dbdate
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  Int64 
 6   test_group  800996 non-null  Int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  Int64 
dtypes: Int64(3), dbdate(1), object(5)
memory usage: 57.3+ MB


#Statistical significance calculation

In [ ]:
# Metric definitions: (numerator_event, denominator_event)
metrics = {
    "add_payment_info/session": ("add_payment_info", "session"),
    "add_shipping_info/session": ("add_shipping_info", "session"),
    "begin_checkout/session": ("begin_checkout", "session"),
    "new_account/session": ("new account", "session")
}

In [ ]:
def calculate_ab_test(df, metrics, group_cols=None, control_group=1, treatment_group=2, significance_level=0.05):

    """
    Calculate AB test metrics and statistical significance for multiple metrics and segments.

    Parameters
    ----------
    df: pd.DataFrame
        Input dataframe with columns: 'test', 'test_group', 'event_name', 'value'
        and optional segmentation columns ('date', 'country', 'device', 'continent', 'channel').
    metrics: dict
        Dictionary mapping metric names to a pair: numerator_event, denominator_event.
        Example: {"add_payment_info/session": ("add_payment_info", "session")}.
    group_cols: None or list, optional
        Columns used for segmentation. If None, only 'test' is used.
        Example: ["device"], ["channel", "country"].
    control_group: int, optional
        Label of the control group in the 'test_group' column.
    treatment_group: int, optional
        Label of the treatment group in the 'test_group' column.
    significance_level: float, optional
        Threshold for statistical significance.

    Returns
    -------
    pd.DataFrame
        Table with one row per metric and segment, containing conversion rates,
        relative change (%), z-statistic, p-value, and the 'significant' flag.
    """

    results = []

    # Ensure that 'test' is always included as the primary grouping column
    if group_cols is None:
        group_cols = ["test"]
    if "test" not in group_cols:
        group_cols = ["test"] + group_cols

    # Get unique segment combinations
    groups = df[group_cols].drop_duplicates()

    # Sort segment groups so tests appear in numerical order
    groups = groups.sort_values("test")

    # Filter the dataset to the current segment (defined by group_cols)
    for _, group_values in groups.iterrows():
        subset = df
        for col in group_cols:
            subset = subset[subset[col] == group_values[col]]

        # Iterate over all configured metrics
        for metric_name, (numerator_event, denominator_event) in metrics.items():

            # Count values for the control group
            numerator_control = subset[
                (subset["event_name"] == numerator_event) & (subset["test_group"] == control_group)
            ]["value"].sum()
            denominator_control = subset[
                (subset["event_name"] == denominator_event) & (subset["test_group"] == control_group)
            ]["value"].sum()

            # Count values for the treatment group
            numerator_test = subset[
                (subset["event_name"] == numerator_event) & (subset["test_group"] == treatment_group)
            ]["value"].sum()
            denominator_test = subset[
                (subset["event_name"] == denominator_event) & (subset["test_group"] == treatment_group)
            ]["value"].sum()

            # Compute conversion rates for both groups
            conversion_test = numerator_test / denominator_test
            conversion_control = numerator_control / denominator_control

            # Skip if control group has zero conversions (avoids division by zero)
            if conversion_control == 0:
                continue

            # Compute relative change in percentage
            relative_change = (conversion_test - conversion_control) / conversion_control * 100

            # Check if both groups satisfy the normal approximation requirement for a Z-test:
            # n * p ≥ 5  and  n * (1 - p) ≥ 5
            normal = (
                denominator_test * conversion_test >= 5 and
                denominator_test * (1 - conversion_test) >= 5 and
                denominator_control * conversion_control >= 5 and
                denominator_control * (1 - conversion_control) >= 5
            )

            # Run Z-test only if normality conditions are satisfied.
            # Otherwise skip this segment because Z-test is statistically invalid.
            if normal:
                z_stat, p_value = proportions_ztest(
                    [numerator_test, numerator_control],
                    [denominator_test, denominator_control]
                )
            else:
                continue

            # Check if the difference is statistically significant
            is_significant = p_value < significance_level

            # Store the computed results for this metric and segment
            row = {
                "metric": metric_name,
                "numerator_event": numerator_event,
                "denominator_event": denominator_event,
                "numerator_test": numerator_test,
                "denominator_test": denominator_test,
                "conversion_test": conversion_test.round(4),
                "numerator_control": numerator_control,
                "denominator_control": denominator_control,
                "conversion_control": conversion_control.round(4),
                "relative_change_%": relative_change.round(2),
                "z_stat": z_stat.round(3),
                "p_value": p_value.round(4),
                "significant": is_significant,
            }

            # Add all grouping fields (e.g. test, device, country) to the result row
            for col in group_cols:
                row[col] = group_values[col]
            results.append(row)

    # Build a DataFrame from all computed result rows
    df_results = pd.DataFrame(results)

    # Reorder columns so that group identifiers appear first
    other_cols = [col for col in df_results.columns if col not in group_cols]
    df_results = df_results[group_cols + other_cols]

    return df_results

In [ ]:
calculate_ab_test(df, metrics, group_cols=["device", "channel"])

,test,device,channel,metric,numerator_event,denominator_event,numerator_test,denominator_test,conversion_test,numerator_control,denominator_control,conversion_control,relative_change_%,z_stat,p_value,significant
0,1,desktop,Social Search,add_payment_info/session,add_payment_info,session,188,2355,0.0798,126,2232,0.0565,41.41,3.134,0.0017,True
1,1,desktop,Social Search,add_shipping_info/session,add_shipping_info,session,246,2355,0.1045,174,2232,0.0780,34.00,3.111,0.0019,True
2,1,desktop,Social Search,begin_checkout/session,begin_checkout,session,303,2355,0.1287,230,2232,0.1030,24.86,2.706,0.0068,True
3,1,desktop,Social Search,new_account/session,new account,session,203,2355,0.0862,164,2232,0.0735,17.32,1.587,0.1124,False
4,1,mobile,Direct,add_payment_info/session,add_payment_info,session,237,4129,0.0574,157,4196,0.0374,53.40,4.293,0.0000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,4,tablet,Paid Search,add_shipping_info/session,add_shipping_info,session,26,660,0.0394,18,654,0.0275,43.13,1.196,0.2317,False
226,4,tablet,Paid Search,begin_checkout/session,begin_checkout,session,68,660,0.1030,41,654,0.0627,64.35,2.651,0.0080,True
227,4,tablet,Paid Search,new_account/session,new account,session,58,660,0.0879,54,654,0.0826,6.43,0.345,0.7304,False
228,4,tablet,Undefined,begin_checkout/session,begin_checkout,session,9,120,0.0750,42,126,0.3333,-77.50,-4.996,0.0000,True


In [ ]:
# Run the A/B testing function and collect results
df_result = calculate_ab_test(df, metrics)

# Save the results as a CSV file in the current working directory
filename = "ab_test_results.csv"
df_result.to_csv(filename, index=False)

#Conclusions

**[ Test 1 ]**

- **Add Payment Info:** `+12.54%`, *significant*  
- **Add Shipping Info:** `+6.56%`, *significant*  
- **Begin Checkout:** `+6.66%`, *significant*  
- **New Account:** `–3.35%`, *not significant*

**Висновок:** тест демонструє послідовне та статистично значуще зростання на основних етапах. Користувачі частіше додають інформацію про оплату й доставку та переходять до checkout, що свідчить про покращення користувацького досвіду та підвищення готовності до покупки. Негативне відхилення у **New Account** не є критичним, оскільки воно не статистично значуще.

---

**[ Test 2 ]**

- **Add Payment Info:** `+3.58%`, *not significant*  
- **Add Shipping Info:** `+1.65%`, *not significant*  
- **Begin Checkout:** `+1.99%`, *not significant*  
- **New Account:** `+1.24%`, *not significant*

**Висновок:** тест демонструє невелике зростання у всіх ключових метриках, проте жодна зі змін не є статистично значущою. Це свідчить про слабкий або відсутній ефект. Попри це, гіпотеза може мати потенціал і може бути використана як основа для майбутніх тестів чи покращених варіацій.

---

**[ Test 3 ]**

- **Add Payment Info:** `+1.47%`, *not significant*  
- **Add Shipping Info:** `–2.62%`, *not significant*  
- **Begin Checkout:** `–3.35%`, *significant*  
- **New Account:** `–1.13%`, *not significant*

**Висновок:** більшість метрик демонструють незначні коливання, однак погіршення у **Begin Checkout** є статистично значущим. Це означає, що зміни негативно впливають на ключовий етап тестування, пов’язаний із переходом до оформлення замовлення. Інші метрики показують незначущі відхилення.

---

**[ Test 4 ]**

- **Add Payment Info:** `–3.54%`, *not significant*  
- **Add Shipping Info:** `–3.41%`, *not significant*  
- **Begin Checkout:** `–2.35%`, *significant*  
- **New Account:** `–3.36%`, *significant*

**Висновок:** тест демонструє чіткий негативний тренд у всіх основних метриках, а два етапи — **Begin Checkout** та **New Account** — мають статистично значуще погіршення. Зміни погіршують як прогрес користувачів у напрямку до покупки, так і створення акаунтів, що підвищує ризики втрати потенційного доходу.

---

**[ Recommendation Across All A/B Tests ]**

- **`Test 1`** — показує стабільне та значуще покращення ключових метрик: **рекомендовано впровадити зміни**.  
- **`Test 2`** — ефект слабкий і статистично незначущий: **не варто впроваджувати**, але гіпотеза може бути корисною у майбутніх варіаціях.  
- **`Test 3`** — зафіксовано значуще падіння **Begin Checkout**: **впроваджувати зміни не рекомендується**.
- **`Test 4`** —  значуще погіршення **Begin Checkout** та **New Account**: **зміни не рекомендовано застосовувати**.

Перегляд результатів по тоталу доступний за посиланням у файлі  [`ab_test_results.csv`](https://drive.google.com/file/d/1Ov2YtjNL8llzh-CDbQz8XoBXdi_Exuu3/view?usp=sharing).


# Tableau Visualization

> [A/B Test Results Dashboard](https://public.tableau.com/views/ABTestResultsDashboard/ABTestToolsDashboard?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)  

На дашборді відображені результати чотирьох A/B-тестів із перевіркою  **рівномірного розподілу груп 50/50** по тоталу, а також у розрізі **країн, континентів, каналів** та **типів девайсів**.  
Додатково представлено **динаміку кількості сесій** у часі для кожного тесту.

Результати розраховані за чотирма метриками:
- **add_payment_info / session**
- **add_shipping_info / session**
- **begin_checkout / session**
- **new_account / session**

Щоб перейти до перегляду результатів, натисніть на відповідне посилання у верхньому лівому куті.  
У таблиці **“Results”** зібрані підсумкові значення для всіх чотирьох метрик. Кольором позначено метрики, де різниця між тестовою та контрольною групами є  **статистично незначущою (p-value ≥ 0.05)**.

